In [ ]:
import os
import secrets
from PIL import Image
from flask import Flask, render_template, redirect, url_for, session, flash, request
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from flask_wtf.file import FileField, FileAllowed
from wtforms import StringField, PasswordField, SubmitField 
from wtforms.widgets import TextArea
from wtforms.validators import InputRequired, Length, ValidationError 
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from datetime import datetime 
import re

app = Flask(__name__)
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config['SECRET_KEY'] = 'c1155c6a351e49eba15c00ce577b259e'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    bio = db.Column(db.Text)
    admin = db.Column(db.Boolean)
    image_file = db.Column(db.String(20), nullable=False, default='default.jpg')
    notes = db.relationship('Note', backref='writer', lazy='dynamic')

class Note(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(25))
    note_body = db.Column(db.String(100))
    note_writer = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    author = db.Column(db.String(255))
    date_created = db.Column(db.TIMESTAMP, default=datetime.utcnow, nullable=False)
    
class RegisterForm(FlaskForm):
    email = StringField("Email", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    username = StringField("用戶名稱（不少於5個英文字）", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("密碼（不少於6個英文字）", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("註冊")

    def validate_username(self, username):
        existing_user_username = User.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("用戶已存在")

    def validate_email(self, email):
        existing_user_email = User.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("此電郵已被登記，請重新註冊。")

class LoginForm(FlaskForm):
    username = StringField("用戶名稱", validators=[InputRequired(), Length(max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("密碼", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder":  "Password"})
    submit = SubmitField("登錄")    
    
class ProfileForm(FlaskForm):
    username = StringField('用戶名稱',
                           validators=[InputRequired(), Length(min=2, max=20)])
    email = StringField("電郵地址", validators=[InputRequired(), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    profile_body = StringField(u'Text', widget=TextArea())
    picture = FileField('更新個人圖片', validators=[FileAllowed(['jpg', 'png'])])
    submit = SubmitField('更新')
    
class NewNoteForm(FlaskForm):
    title = StringField("標題", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "標題"})
    note_body = StringField(u'內文', widget=TextArea())
    submit = SubmitField("新增貼文")

class UpdateNoteForm(FlaskForm):
    title = StringField("標題", validators=[InputRequired(), Length(max=25)], render_kw={"placeholder": "標題"})
    note_body = StringField(u'內文', widget=TextArea())
    submit = SubmitField("更新貼文")
    
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


####################################################################################################################
@app.route('/home')
@app.route('/')
def home():
    return render_template('index.html', title='Home')

@app.route("/tabehondai")
def tabehondai():
    return render_template('tabehondai.html')

@app.route("/japan")
def japan():
    return render_template('japan.html')

@app.route("/korean")
def korean():
    return render_template('korean.html')

@app.route("/other")
def other():
    return render_template('other.html')

@app.route("/below200")
def below200():
    return render_template('below200.html')

@app.route("/above200")
def above200():
    return render_template('above200.html')

@app.route("/hongkong")
def hongkong():
    return render_template('hongkong.html')

@app.route("/kowloon")
def kowloon():
    return render_template('kowloon.html')

@app.route("/nt")
def nt():
    return render_template('nt.html')

@app.route("/monthly")
def monthly():   
    return render_template('monthly.html')

@app.route("/year")
def year():
    return render_template('year.html')

@app.route("/recommend")
def recommend():
    return render_template('recommend.html')

@app.route("/about")
def about():
    return render_template('about.html')

@app.route("/contact")
def contact():
    return render_template('contact.html')

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404


##################################################################################################################
@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("new_note"))

        flash("User does not exist, or invalid username or password.")
    return render_template('login.html', title="Login", form=form)

@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = User(username=form.username.data, email=form.email.data, password=hashed_password,bio='',admin=0)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='Register', form=form)

@app.route('/logout', methods=["GET","POST"])
def logout():
    session.clear()
    logout_user()
    return redirect(url_for('login'))

@app.route('/new-note', methods=['GET','POST'])
@login_required
def new_note():
    form = NewNoteForm()
    if form.validate_on_submit():
        new_note = Note(title=form.title.data, note_body=form.note_body.data, writer=current_user, author=current_user.username)
        db.session.add(new_note)
        db.session.commit()
        return redirect(url_for('blogs'))
    return render_template('new_note.html', title='New Note', form=form)


@app.route('/my-notes', methods=['GET','POST'])
@login_required
def view_notes():
    print('current user id:', current_user.id)
    if current_user.admin:
        sqlalchemyObj = db.engine.execute('select note.id, note.title, note.note_body, note.date_created, user.username, user.image_file from note INNER JOIN user ON note.note_writer = user.id ORDER BY user.id')
    else:
        sql = 'select note.id, note.title, note.note_body, note.date_created, user.username, user.image_file from note INNER JOIN user ON note.note_writer = user.id WHERE note.note_writer == '+ str(current_user.id) + ' ORDER BY user.id'
        sqlalchemyObj = db.engine.execute(sql)
    notes = []
    for i in sqlalchemyObj:
        notes.append(i)
        dataset=[]
        dict={}
        for i in notes:
            dict['id'] = i[0]
            dict['title'] = i[1]
            dict['note_body'] = i[2]
            dict['date_created'] = i[3]
            dict['username'] = i[4]
            dict['image_file'] = i[5]

            dataset.append(dict.copy())
    print(dataset)
    return render_template('my_notes.html', notes=dataset, title='My Notes')

@app.route('/blogs', methods=['GET','POST'])
def blogs():
    sqlalchemyObj = db.engine.execute('select note.title, note.note_body, note.date_created, user.username, user.image_file from note INNER JOIN user ON note.note_writer = user.id ORDER BY user.id')
    notes = []
    for i in sqlalchemyObj:
        notes.append(i)
    # print(venues)
    dataset=[]
    dict={}
    for i in notes:
        dict['title'] = i[0]
        dict['note_body'] = i[1]
        dict['date_created'] = i[2]
        dict['username'] = i[3]
        dict['image_file'] = i[4]
        # print(i)
        # print(dict)
        dataset.append(dict.copy())
    print(dataset)
    return render_template('blogs.html', notes=dataset, title='Blog Posts')

@app.route("/profile", methods=['GET', 'POST'])
@login_required
def profile():
    form = ProfileForm()
    if form.validate_on_submit():
        if form.picture.data:
            picture_file = save_picture(form.picture.data)
            current_user.image_file = picture_file
        current_user.username = form.username.data
        current_user.email = form.email.data
        current_user.bio = form.profile_body.data
        db.session.commit()
        flash('Your profile has been updated!', 'success')
        return redirect(url_for('profile'))
    elif request.method == 'GET':
        form.username.data = current_user.username
        form.email.data = current_user.email
        form.profile_body.data = remove_html_tags(current_user.bio)
        if current_user.image_file is None:
            current_user.image_file = 'default.png'
    image_file = url_for('static', filename='profile_pics/' + current_user.image_file)
    return render_template('profile.html', title='Profile', image_file=image_file, form=form)

def save_picture(form_picture):
    random_hex = secrets.token_hex(8)
    _, f_ext = os.path.splitext(form_picture.filename)
    picture_fn = random_hex + f_ext
    picture_path = os.path.join(app.root_path, 'static/profile_pics', picture_fn)

    output_size = (125, 125)
    i = Image.open(form_picture)
    i.thumbnail(output_size)
    i.save(picture_path)

    return picture_fn
    

@app.route('/delete-note/<int:note_id>', methods=['GET',"POST"])
@login_required
def delete_note(note_id):
    note = Note.query.get_or_404(note_id)
    db.session.delete(note)
    db.session.commit()
    return redirect(url_for('view_notes'))

@app.route('/update-note/<int:note_id>', methods=['GET',"POST"])
@login_required
def update_note(note_id):
    form = UpdateNoteForm()
    note = Note.query.get_or_404(note_id)
    if request.method == 'GET':
        form.title.data = note.title
        form.note_body.data = note.note_body
    if form.validate_on_submit():  # request.method="POST"
            note.query.filter_by(id=note_id).first()
            if note:
                note.title = form.title.data
                note.note_body = form.note_body.data
                db.session.commit()
                print("Note id=",note_id)
                print("Updated.")
                return redirect(url_for("view_notes"))

    return render_template('update_note.html', form=form, title='Update Note', id=note_id)


##################################################
if __name__ == '__main__':
    app.run('localhost', 9007)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9007/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Dec/2021 15:26:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 15:26:59] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 15:27:01] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 15:27:06] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [12/Dec/2021 15:27:06] "GET /new-note HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 15:27:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 15:40:42] "GET /new-note HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 15:40:43] "GET /login HTTP/1.1" 200 -
